In [ ]:
!wget -q https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, RegexpTokenizer, TweetTokenizer, PorterStemmer


from re import sub
import tokenization
from tokenization import FullTokenizer
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
df_train= pd.read_csv('../input/amz-data/dataset/train.csv', escapechar = "\\", quoting = 3)
df_mini= df_train[:40000]

df_test= pd.read_csv('../input/amz-data/dataset/test.csv', escapechar = "\\", quoting = 3)

df= pd.concat([df_train, df_test])
df.reset_index(inplace=True)

print(df_train.shape, df_test.shape, df.shape)

In [ ]:
df_mini.head(3)

In [ ]:
df_train.isna().sum(), df_test.isna().sum()

In [ ]:
msno.matrix(df_mini)

In [ ]:
df =df.replace(np.nan, 'error', regex=True)
df.isna().sum()

In [ ]:
df.head(5)

## NLP Processing

In [ ]:
def remove_URL(text):
    url = RegexpTokenizer(r'https?://\S+|www\.\S+', gaps = True)
    return " ".join(url.tokenize(text))

def stopWords(tweet):  
    stop_words, toker = stopwords.words('english'), TweetTokenizer()
    words_tokens = toker.tokenize(tweet)
    return " ".join([word for word in  words_tokens if not word in stop_words])

def remove_pontucations(text):
    tokenizer_dots = RegexpTokenizer(r'\w+')
    return " ".join(tokenizer_dots.tokenize(text))

def remove_words_min(text, param):
    tmp = text
    
    for x in tmp.split():
        if len(x) < param:
            tmp = tmp.replace(x, '')
    return " ".join(tmp.split())

In [ ]:
def clean(data, col='TITLE'):                                          #print('make text lowercase')
    data[col] = data[col].apply(lambda x: x.lower())                   #print('delete excess spaces')
    data[col] = data[col].apply(lambda x: " ".join(x.split()))         #print('delete numbers')
    data[col] = data[col].apply(lambda x: sub(r'\d+', '', x))          #print('remove punctuation and special character')
    data[col] = data[col].progress_apply(lambda x: remove_pontucations(x))      #print('remove stopWords')
    data[col] = data[col].progress_apply(lambda x: stopWords(x))       #print('remove _ attatched words')
    data[col] = data[col].apply(lambda x: x.replace('_', ' '))
    #data[col] = data[col].apply(lambda x: remove_words_min(x, 2))

In [ ]:
# BEFORE 
print(df.TITLE[0])
print(df.DESCRIPTION[0])
print(df.BULLET_POINTS[0])
print(df.BRAND[0])

In [ ]:
clean(df, col='TITLE')
clean(df, col='DESCRIPTION')
clean(df, col='BULLET_POINTS')
clean(df, col='BRAND')

In [ ]:
# AFTER 
print(df.TITLE[0])
print(df.DESCRIPTION[0])
print(df.BULLET_POINTS[0])
print(df.BRAND[0])

In [ ]:
df.to_csv('amz_df.csv', index=False)

In [ ]:
df_train.BROWSE_NODE_ID.unique().shape

In [ ]:
plt.bar(df_train.BROWSE_NODE_ID.value_counts().index, df_train.BROWSE_NODE_ID.value_counts())
plt.show()

In [ ]:
sns.kdeplot(data=df_train, x="BROWSE_NODE_ID")
plt.show()

## EMB. MOdel

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

In [ ]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
df.TITLE[0:3].values

In [ ]:
hub_layer(df.TITLE[0:3].values)